# Biomag 2016 - Competition 3

 *Alexandre Barachant, Jean-Remi King*

## Challenge description

The goal of this challenge was to discriminate the Magneto-Encephalography (MEG) responses to happy vs non-happy faces recorded from four subjects with 204 planar gradiometers. The original data were sampled at 1kHz, filtered between 0.1 and 40Hz and finally down-sampled to 125Hz.

Six possible facial expressions could be presented to the subject. The subject's task was to press a button whenever a happy face was presented.

Therefore, the MEG activity of interest is likely to be dominated by 1) the event-related field (ERF) specific to happy faces and 2) the neural activity induced by the following button press. 

The data was split into a training set and a test set. No labels were provided for the test set.

In the training set, we identified that the pictures were grouped by randomly shuffled sequences of size 12, with two pictures of each category within each group. We inferred that the same regularity existed in the testing set.

## Approach

Our approach consisted in ensembling three classification pipelines built to extract both evoked and induced responses. This work entirely relies on function already availables in the toolboxes [MNE-python](http://mne-tools.github.io/stable/index.html), [pyRiemann](http://pythonhosted.org/pyriemann/index.html) and [Scikit-Learn](http://scikit-learn.org/).


In [39]:
import numpy as np
import pandas as pd
from collections import OrderedDict

from scipy.io import loadmat

from mne.decoding import UnsupervisedSpatialFilter, Vectorizer
from sklearn.decomposition import PCA
from sklearn.cross_validation import KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace
from pyriemann.estimation import ERPCovariances, XdawnCovariances, HankelCovariances
from pyriemann.utils.base import _matrix_operator



def epoch_data(data, window=125, offset=0):
    """Epoch data"""
    MEG, trigger = data['planardat'], data['triggers']

    X, y = list(), list()
    
    trials = np.r_[trigger.t1, trigger.t2, trigger.t3,
                   trigger.t4, trigger.t5, trigger.t6]

    values = np.array([0]*len(trigger.t1) + [1]*len(trigger.t2) +
                      [2]*len(trigger.t3) + [3]*len(trigger.t4) +
                      [4]*len(trigger.t5) + [5]*len(trigger.t6))

    # Epoch training set
    ix = np.argsort(trials)
    trials, values = trials[ix], values[ix]
    for ii, start in enumerate(trials):
        X.append(MEG[:, slice(start + offset, start + window + offset)])
        y.append(values[ii])

    # Epoch testing set
    X_test = list()
    for t in trigger.test:
        sl = slice(t + offset, t + window + offset)
        # Stack zeros on last trials in case window is too long
        epoch = np.zeros((len(MEG), window))
        epoch[:, :len(MEG[0, sl])] = MEG[:, sl]
        X_test.append(epoch)

    # Format
    X = 1e12 * np.array(X)
    X_test = 1e12 * np.array(X_test)
    y = np.array(y) == 3

    return X, y, X_test


def local_debias(y_preds):
    """The sum of each group of 12 trials must be 1."""
    y_preds = np.array(y_preds)
    y_preds = np.reshape(y_preds, (-1, 12))
    y_preds /= np.sum(y_preds, 1)[:, np.newaxis]
    return y_preds.ravel()



## Classification Pipelines

The 3 pipelines rely on Riemannian geometry classifiers fitted on the covariance matrices of each trial. More specifically, these pipeline made use of the tangent space mapping described in [1, 2] followed by a logistic regression. The main difference between these pipeline resides in the definition of the features space and in the estimation of the covariance matrices. 

#### Covariance estimation
Let $\mathbf{X}_i \in \Re^{C \times N}$ denote an epoch (trial) of index $i$ with $C$ the number of channels and $N$ the number of time samples. Let be $y_i$ be the class (target (happy) or non-target (not happy)) of $\mathbf{X}_i$. 
The spatial covariance matrix $\mathbf{\Sigma}_i \in \Re^{C \times C}$ of $\mathbf{X}_i$ is estimated using the sample covariance matrix estimator (SCM):

$$\mathbf{\Sigma}_i = \frac{1}{N} \left( \mathbf{X}_i - \mathbf{\bar{x}}_i \right) \left( \mathbf{X}_i - \mathbf{\bar{x}}_i \right)^T$$


where $\mathbf{\bar{x}}_i \in \Re^{C}$ is the average of the trial across time. For each of the 3 pipelines, the covariance estimation only varies in the way $\mathbf{X}_i$ is built. Additionnaly, to ensure that the matrices are symmetric and positive definite (a requirement to use Riemannian geometry), regularization can be applied on the estimated covariances matrices. In this work, the Ledoit-wolf Shrinkage [4] or the Oracle Approximation Shrinkage [5] will be used.

#### Tangent Space mapping
Riemannian geometry provides a natural way to manipulate and measure difference between SPD matrices. In this work, we used tangent space mapping as a way to take into account the manifold structure while having a vector-representation of the matrices. For any covariance matrix $\mathbf{\Sigma}_i$, we define its "tangent vector" $\mathbf{s}_i \in \Re^{C(C+1)/2}$ :

$$\mathbf{s}_i = \mathrm{upper} \left( \log \left( \mathbf{\Sigma}_\mu^{-1/2} \mathbf{\Sigma}_i  \mathbf{\Sigma}_\mu^{-1/2} \right) \right), $$ 

where $\log$ is the matrix logarithm, $\mathrm{upper}$ is the operator consiting in taking the upper triagular part of the matrix, applying a coefficient $\sqrt{2}$ on its off-diagonal elements, and vectorizing the results. Finaly $\mathbf{\Sigma}_\mu$ is also a covariance matrix, defining the reference point of the tangent space (its origin in the manifold). In this work, this reference point was chosen to be the log-Euclidean mean of all training data:

$$\mathbf{\Sigma}_\mu = \exp \left( \frac{1}{N}\sum_i^N \log(\mathbf{\Sigma}_i ) \right)$$

#### Local "debiasing"

Since the experimental design was not trully randomized, but consisted in shuffled sequences of 12 faces (with two target in each group), we applied a post-processing step on the predictions. This step consisted in ensuring that the sum of probabilities of each group of 12 trials was 1.

#### Ensembling and Bagging

The ensembling of the 3 pipeline is simply done by the summation of their individual predictions. To further improve the robustness of the classification, and to reduce overfitting, we also ensemble results for different set of epoching offset : 10, 20, 30, 40 and 50 time samples with a constant window of 150 time samples.

We now turn to the specificities of each pipeline.

### ERPCov

This first pipeline is made of
- a decomposition of the spatial filters with a Principal Component Analysis (PCA) for dimensionality reduction (70 components)
- an estimation of the ERF Covariance to capture both evoked and induce responses
- a Common Spatial Pattern (CSP) to reduce the dimensionality of the covariance matrices (30 components)
- a mapping to the tangent space (with log-euclidean mean as reference point) to meaningfully vectorize the data
- and a penalized logistic regression for the final classification stage.

The classic estimation of a spatial covariance does not take into account the order of the samples. The ERPCov pipeline relies on a special form of covariance matrix estimation that embeds the temporal information about the evoked response [3]. This estimation consists in the concatenation, along the channel axis, of the averaged ERF of each class before estimating the spatial covariance matrix:

$$\mathbf{\tilde{X}}_i = 
\left[ \begin{array}{c}
\mathbf{P}_t \\
\mathbf{P}_{nt} \\
\mathbf{X}_i
\end{array} \right] $$

where $\mathbf{P}_t$ and $\mathbf{P}_{nt}$ are the averaged (across trial) evoked response for the target and non-target class, respectively.

the total dimension of the matrices is $\tilde{C} = 3 \times C = 3 \times 70 = 210$. The resulting covariance matrix is thus composed by the cross-covariance of the trial with the prototypical evoked response of each class and by the standard covariance of the trial. 

Consequently, if the trial $\mathbf{X}_i$ contains a target evoked response synchronized to the prototypical target response, then it will produce a specific cross-covariance structure with the target prototype. In addition, if the trial contains a task-related induced activity, the presence of the sample covariance matrice of $\mathbf{X}_i$ will allow its detection. Therefore, this type of covariance estimation makes use of both evoked and induced responses within the same pipeline.

After estimating the covariance matrices, a CSP is applied to reduce their dimension to 30 components, projected in the tangent space and classified with a logistic regression.

In [40]:
clfs = OrderedDict()
clfs['ERPCov'] = make_pipeline(UnsupervisedSpatialFilter(PCA(70), average=False),
                               ERPCovariances(estimator='lwf'),
                               CSP(30, log=False),                           
                               TangentSpace('logeuclid'),
                               LogisticRegression('l2'))

### XdawnCov

This pipeline is composed by 
- PCA for dimensionality reduction (50 components)
- XdawnERP Covariance estimation
- Tangent space mapping (with log-euclidean mean as reference point)
- Logistic Regression for the final classification stage.

The XdawnCov pipeline is similar to ERPCov, except that a Xdawn spatial filtering [6] is applied before the covariance estimation in order to increase the SNR of the ERP response.

For each of the two classes (target and non-target), a set of Xdawn spatial filters are estimated and applied on the data. Let $\mathbf{V}_{t}$ and $\mathbf{V}_{nt}  \in \Re^{C \times K}$ be the spatial filters corresponding to the class target and non target, respectively, then we get:

$$\mathbf{\tilde{X}}_i = 
\left[ \begin{array}{c}
\mathbf{V}_{t}^T \mathbf{P}_t \\
\mathbf{V}_{nt}^T \mathbf{P}_{nt} \\
\mathbf{V}_{t}^T \mathbf{X}_i\\
\mathbf{V}_{nt}^T \mathbf{X}_i
\end{array} \right] $$

In this pipeline, $K=12$ Xdawn filters for each class are used, therefore the total dimention of the covariance matrices is $\tilde{C} = 4\times K= 48$. 

After estimating these covariances matrices, they are projected on the Riemannian tangent space and classified with a l2-regularized logistic regression.

In [41]:
clfs['XdawnCov'] = make_pipeline(UnsupervisedSpatialFilter(PCA(50), average=False),
                                 XdawnCovariances(12, estimator='lwf', xdawn_estimator='lwf'),
                                 TangentSpace('logeuclid'),
                                 LogisticRegression('l2'))

### HankelCov

This pipeline is made of
- PCA for dimensionality reduction (70 components),
- Hankel Covariance estimation (delays 1, 8, 12 and 64 time samples)
- CSP for reduction of covariance matrices (15 components)
- Tangent space mapping (with log-euclidean mean as reference point)
- Logistic Regression for the final classification stage.

This estimation is obtained by concatenating multiple time-lagged versions of each trial. This estimation detects the auto-correlation and the cross-correlation between the channels. It is similar to the Common Spatio-Spectral Pattern algorithm. 

$$\mathbf{\tilde{X}}_i = 
\left[ \begin{array}{c}
\delta(n_1, \mathbf{X}_i) \\
\vdots \\
\delta(n_j, \mathbf{X}_i)\\
\vdots \\
\delta(n_J, \mathbf{X}_i)
\end{array} \right] $$

where $\delta(n_j, \mathbf{X}_i)$ is an opperator that add a lag of $n_j$ to the data $\mathbf{X}_i$. In the case, we chose a logaritmicaly spaced set of five lags: 0, 1, 8, 12, 64. The dimension of the covariance matrices is therefore $\tilde{C} = 5 * C = 350$.
A CSP was then applied to reduce the dimensionality of the covariance matrices before projecting them to the tangent space, and classifying these vectors with a penalized logistic regression.

In [42]:
clfs['HankelCov'] = make_pipeline(UnsupervisedSpatialFilter(PCA(70), average=False),
                                  HankelCovariances(delays=[1, 8, 12, 64], estimator='oas'),
                                  CSP(15, log=False),
                                  TangentSpace('logeuclid'),
                                  LogisticRegression('l2'))

## Cross-Validation 

We first evaluated these classifiers on the training set using a 10-fold cross-validation scheme.

In [55]:
offsets = [10, 20, 30, 40, 50]
results = pd.DataFrame(index=range(1,5), columns=clfs.keys() + ['Ensemble'])
for subject in range(1, 5):
    # Load the data
    data = loadmat('./data/meg_data_%da.mat' % subject,
                   squeeze_me=True, struct_as_record=False)
    
    preds = np.zeros((240, len(clfs)))

    for offset in offsets:
        # Epoching
        X, y, _ = epoch_data(data, window=150, offset=offset)  

        # define CV and prediction
        cv = KFold(len(y), n_folds=10, shuffle=False, random_state=4343)

        # iterate over models and train/test partitions
        for jj, clf in enumerate(clfs):
            for train, test in cv:
                clfs[clf].fit(X[train], y[train])

                # get the predictions
                pr = clfs[clf].predict_proba(X[test])[:, -1]
                preds[test, jj] += local_debias(pr)

            results.loc[subject, clf] = roc_auc_score(y, preds[:, jj])
    results.loc[subject, 'Ensemble'] = roc_auc_score(y, local_debias(preds.mean(1)))

results.loc['Mean', :] = results.mean()
results

,ERPCov,XdawnCov,HankelCov,Ensemble
1,0.999125,0.997125,0.99975,0.99925
2,0.909625,0.917875,0.904875,0.92925
3,0.990875,0.9875,0.979125,0.994625
4,0.98875,0.976,0.99325,0.992875
Mean,0.9720937,0.969625,0.96925,0.979


The best pipeline is ERPCov with an average AUC of 0.972. The ensemble of the 3 classifiers leads to an AUC score of 0.979.

## Test predictions

We now apply each of the ensembling of these pipelines to the test data:

In [56]:
offsets = [10, 20, 30, 40, 50]
for subject in range(1, 5):
    data = loadmat('./data/meg_data_%da.mat' % subject, squeeze_me=True, struct_as_record=False)
    preds = np.zeros(240)
    for offset in offsets:
        # Epoching
        X, y, X_test = epoch_data(data, window=150, offset=offset)  
        for jj, clf in enumerate(clfs):
            clfs[clf].fit(X, y)
            preds += local_debias(clfs[clf].predict_proba(X_test)[:, -1])
    preds = local_debias(preds)    
    results = pd.DataFrame(data=preds, columns=['Predictions'])
    results.to_csv('predictions_Subject%d.csv' % subject)
    print('Subject%d Done !' % subject)

Subject1 Done !
Subject2 Done !
Subject3 Done !
Subject4 Done !


## Conclusion 

The accuracy seems lowest for the second subject, but is almost perfect for the 3 other. Because the decoding of the happy face is confounded with the behavior of the subject, it is plausible that the decoder gets biased by the subject's responses, even when they are not correct.

From a neuroscientific perspective, the decoding of the present dataset highlights the importance of
 - 1) truly randomizing the stimuli. Here our analyses break the independence across samples, which would make subsequent statistical estimations more complex. This is all the more true, if users start to build very opaque pipelines that may benefit from trivial inter-subject information.
 - 2) adequately decorrelating the experimental factors. Here, we cannot interpret whether the decoding is due to a sensory and/or a motor response although it is likely to be critical for the tested hypotheses.

Overall, we believe these competitions offers a great way to evaluate and disseminate state-of-the-art and original methods and thank the organizers to have taken the time to make it possible. 

In the future, we recommend making a leader-board which would motivate the competitors in pushing their methods to their limits.


## References 

> [1] A. Barachant, S. Bonnet, M. Congedo and C. Jutten, “Multiclass Brain-Computer Interface Classification by Riemannian Geometry,” in IEEE Transactions on Biomedical Engineering, vol. 59, no. 4, p. 920-928, 2012. [pdf](http://hal.archives-ouvertes.fr/docs/00/68/13/28/PDF/Barachant_tbme_final.pdf)
>
> [2] A. Barachant, S. Bonnet, M. Congedo and C. Jutten, “Classification of covariance matrices using a Riemannian-based kernel for BCI applications“, in NeuroComputing, vol. 112, p. 172-178, 2013. [pdf](http://hal.archives-ouvertes.fr/docs/00/82/04/75/PDF/BARACHANT_Neurocomputing_ForHal.pdf)
>
> [3] A. Barachant, M. Congedo ,"A Plug&Play P300 BCI Using Information Geometry", arXiv:1409.0107. [link](http://arxiv.org/abs/1409.0107)
>
> [4] O. Ledoit and M. Wolf, “A Well-Conditioned Estimator for Large-Dimensional
Covariance Matrices”, Journal of Multivariate Analysis, Volume 88, Issue 2, February 2004, pages 365-411.
>
> [5] Chen et al., “Shrinkage Algorithms for MMSE Covariance Estimation”,
IEEE Trans. on Sign. Proc., Volume 58, Issue 10, October 2010.
>
> [6] Rivet, B., Souloumiac, A., Attina, V., & Gibert, G. (2009). xDAWN algorithm to enhance evoked potentials: application to brain–computer interface. IEEE Transactions on Biomedical Engineering, 56(8), 2035-2043.